## Variable selection using sRMMD knockoffs on real dataset
This notebook apply sRMMD-knockoff filter on 'C18 positive' metabolites dataset to find important biomarkers. Before applying the knockoff filter, severeal preprocessing steps- missing value impuation, interquartile range filter and standardization are done on the data. After training the generator with the preprocessed dataset, knockoffs are produced 100 times and Random forest is applied to find the knockoff statistic for each instance. Metabolites that are found important (after applying the knockoff filter) more than 70 are only considered.   

In [ ]:
import numpy as np
import pandas as pd
import gzip
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from gaussian import GaussianKnockoffs
from machine import KnockoffGenerator
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import iqr
from sklearn.preprocessing import StandardScaler

In [2]:
def missing_value_imputation(data):
    x = data.to_numpy()
    imputer = KNNImputer(n_neighbors = 2, weights="uniform")
    imputed_x =  imputer.fit_transform(x)
    new_data = data.copy()
    new_data.iloc[:, :] = imputed_x
    return new_data

def rf_oob_score(X, X_knockoff, y):
    
    p = X.shape[1]
    clf = RandomForestClassifier(n_estimators = 500, bootstrap = True, oob_score = True, max_features = 'sqrt')
    clf.fit(np.hstack((X, X_knockoff)), y)
    Z = clf.feature_importances_
    W = np.abs(Z[:p]) - np.abs(Z[p : (2 * p)])
    return Z, W

def kfilter(W, offset = 0.0, q = 0.1):
    t = np.insert(np.abs(W[W != 0]), 0, 0) # omitting 0 value and then add zero at the begining
    t = np.sort(t)
    ratio = np.zeros(len(t));
    for i in range(len(t)):
        ratio[i] = (offset + np.sum(W <= -t[i])) / np.maximum(1.0, np.sum(W >= t[i]))
        
    index = np.where(ratio <= q)[0]
    if len(index)== 0:
        thresh = float('inf')
    else:
        thresh = t[index[0]]
       
    return thresh

def selection(W, offset = 0.0, nominal_fdr = 0.1):
    
    W_threshold = kfilter(W, q = nominal_fdr)
    selected = np.where(W >= W_threshold)[0]
    return selected, W_threshold

In [3]:
df =  pd.read_csv('dataset/Real dataset/C18_Negative.txt', sep='	', dtype = object)
df = df.rename(columns = df.iloc[0])
df = df.drop(df.index[0]).reset_index(drop = True)
df.columns = df.columns.str.replace(' ', '').str.lower()
diagnose = df.columns[1:].str.split('|').str[0]
diagnose= diagnose.str.split(':').str[1]
_, y = np.unique(diagnose, return_inverse=True)
df = df.set_index('factors')
print('unique diagnosis', diagnose.unique(), '\n Metabolite X Samples: ' , df.shape)

unique diagnosis Index(['cd', 'uc', 'nonibd'], dtype='object') 
 Metabolite X Samples:  (91, 546)


In [4]:
T = [0.8]
data =  df.copy().T
no_of_samples =  data.shape[0]
thresh = int(no_of_samples * T[0])
data = data.dropna(axis = 1, thresh = thresh)#keeping the metabolites which has atleast 70% percenet filled values
missing_percentage_after = data.isnull().sum().sum()/ (data.shape[0] * data.shape[1])
print('After: (Samples X Metabolites): ' , data.shape, '\t\t percentage of missing values: %.3f'
      %missing_percentage_after,'\n')

After: (Samples X Metabolites):  (546, 80) 		 percentage of missing values: 0.009 



In [5]:
imputed_data = missing_value_imputation(data)
print('After: (Samples X Metabolites): ' , data.shape)

After: (Samples X Metabolites):  (546, 80)


In [6]:
standard_data = imputed_data.copy()
standard_data.iloc[:, :] = StandardScaler().fit_transform(standard_data)

In [7]:
xTrain = standard_data.values
n, d = xTrain.shape 

In [8]:
print("Generated a training dataset of size: %d x %d." %(xTrain.shape))
SigmaHat = np.cov(xTrain, rowvar=False)
second_order = GaussianKnockoffs(SigmaHat, mu=np.mean(xTrain,0), method="sdp")
corr_g = (np.diag(SigmaHat) - np.diag(second_order.Ds)) / np.diag(SigmaHat)
print('Average absolute pairwise correlation: %.3f.' %(np.mean(np.abs(corr_g))))

Generated a training dataset of size: 546 x 80.
Average absolute pairwise correlation: 0.657.


In [11]:
pars={"epochs":100, 
      "epoch_length": 20, 
      "d": d,
      "dim_h": int(d*d),
      "batch_size": int(0.5*n), 
      "lr": 0.01, 
      "lr_milestones": [100],
      "GAMMA": 1,
      "losstype": 'sRMMD',
      "epsilon":50,
      "target_corr": corr_g,
      "sigmas":[1.,2.,4.,8.,16.,32.,64.,128.]
     }

In [12]:
machine = KnockoffGenerator(pars)
machine.train(xTrain)

[   1/ 100], Loss: 0.1500, sRMMD : 0.1122, Decorr: 0.619
[   2/ 100], Loss: 0.1091, sRMMD : 0.0999, Decorr: 0.589
[   3/ 100], Loss: 0.0813, sRMMD : 0.0751, Decorr: 0.638
[   4/ 100], Loss: 0.0810, sRMMD : 0.0752, Decorr: 0.626
[   5/ 100], Loss: 0.0829, sRMMD : 0.0751, Decorr: 0.615
[   6/ 100], Loss: 0.0585, sRMMD : 0.0531, Decorr: 0.616
[   7/ 100], Loss: 0.0839, sRMMD : 0.0777, Decorr: 0.608
[   8/ 100], Loss: 0.0651, sRMMD : 0.0600, Decorr: 0.630
[   9/ 100], Loss: 0.0580, sRMMD : 0.0528, Decorr: 0.623
[  10/ 100], Loss: 0.0632, sRMMD : 0.0579, Decorr: 0.624
[  11/ 100], Loss: 0.0719, sRMMD : 0.0666, Decorr: 0.631
[  12/ 100], Loss: 0.0829, sRMMD : 0.0757, Decorr: 0.633
[  13/ 100], Loss: 0.0608, sRMMD : 0.0563, Decorr: 0.632
[  14/ 100], Loss: 0.0576, sRMMD : 0.0534, Decorr: 0.619
[  15/ 100], Loss: 0.0639, sRMMD : 0.0591, Decorr: 0.599
[  16/ 100], Loss: 0.0552, sRMMD : 0.0498, Decorr: 0.619
[  17/ 100], Loss: 0.0612, sRMMD : 0.0569, Decorr: 0.628
[  18/ 100], Loss: 0.0562, sRMM

In [22]:
repeat_exp = 100
indices = []
for i in range(repeat_exp):
    xKnockoff = machine.generate(xTrain)
    Z, W = rf_oob_score(xTrain, xKnockoff, y)
    S, T = selection(W, offset =  1.0, nominal_fdr =  0.05)# original offset = 1.0
    indices.append(list(S))
    if (i%10)==0: print('no of Simulation completed:', i + 1)

no of Simulation completed: 1
no of Simulation completed: 11
no of Simulation completed: 21
no of Simulation completed: 31
no of Simulation completed: 41
no of Simulation completed: 51
no of Simulation completed: 61
no of Simulation completed: 71
no of Simulation completed: 81
no of Simulation completed: 91


In [23]:
indices_flatten = [j for sub in indices for j in sub]
selected =  np.unique(indices_flatten)
count_df = pd.DataFrame()
unique, counts = np.unique(indices_flatten, return_counts = True)
count_df['name'] = unique
count_df['count'] = counts
count_df = count_df.sort_values(by = ['count'], ascending = False)
count_df.drop(count_df[count_df['count'] < 70].index, inplace = True)
selected = count_df['name'].to_numpy()
meta_list = list(standard_data.columns[selected])
meta_list

['salicylate',
 'phenyllactate',
 '9.10-diHOME',
 '1.2.3.4-tetrahydro-beta-carboline-1.3-dicarboxylate',
 '12.13-diHOME',
 '4-hydroxystyrene',
 'saccharin',
 'porphobilinogen',
 'p-hydroxyphenylacetate',
 'hydrocinnamate',
 'taurolithocholate',
 'urobilin',
 'adrenate',
 'carboxyibuprofen',
 'docosapentaenoate',
 'caproate',
 'olmesartan',
 'masilinate',
 'mandelate']